In [2]:
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries

C:\Users\adevr\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
APIkey = "2SRLF3NBCPEO7UI5"

from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key=APIkey, output_format="pandas")
data, _ = ts.get_daily(symbol="TSLA", outputsize="full")
data = data.loc[
    "2012-01-01":"2021-12-31"
]  # Let's limit our data to the period 2012-2021
data.tail()

KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'